In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.metrics import classification_report, precision_recall_curve, auc
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

import torch 
import os
import torch

import warnings
warnings.filterwarnings("ignore")

In [2]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

print(device)

cuda


In [4]:
## Load preprocess data
data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/VAE-CTAB-GAN/preprocess/processed.csv"
data = pd.read_csv(data_path, low_memory=False)

In [6]:
data.describe()

,0,1,2,3,4,5,6,7,8,9,...,269,270,271,272,273,274,275,276,277,278
count,174231.000000,174231.000000,174231.000000,174231.000000,174231.000000,174231.000000,174231.000000,174231.000000,174231.000000,174231.000000,...,174231.000000,174231.000000,174231.000000,174231.000000,174231.000000,174231.000000,174231.000000,174231.000000,174231.0,174231.0
mean,0.001067,0.132606,0.122854,0.114274,0.089513,0.089353,0.073288,0.073029,0.070653,0.062968,...,0.037295,0.034902,0.034248,0.026832,0.016604,0.011640,0.599193,0.400807,0.0,1.0
std,0.235172,0.339149,0.328271,0.318144,0.285484,0.285253,0.260609,0.260186,0.256246,0.242906,...,0.189485,0.183532,0.181865,0.161593,0.127784,0.107258,0.490063,0.490063,0.0,0.0
min,-0.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0
25%,-0.128023,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0
50%,-0.001399,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,1.0
75%,0.123392,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.0,1.0
max,0.990000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.0


In [8]:
import pickle

with open("preprocess/transformer/transformer.pkl", "rb") as f:
    transformer = pickle.load(f)

for i, info in enumerate(transformer.output_info):
    print(f"{i}번째 컬럼: dim={info[0]}, activation={info[1]}")


0번째 컬럼: dim=1, activation=tanh
1번째 컬럼: dim=15, activation=softmax
2번째 컬럼: dim=1, activation=tanh
3번째 컬럼: dim=14, activation=softmax
4번째 컬럼: dim=1, activation=tanh
5번째 컬럼: dim=10, activation=softmax
6번째 컬럼: dim=1, activation=tanh
7번째 컬럼: dim=16, activation=softmax
8번째 컬럼: dim=1, activation=tanh
9번째 컬럼: dim=15, activation=softmax
10번째 컬럼: dim=1, activation=tanh
11번째 컬럼: dim=12, activation=softmax
12번째 컬럼: dim=1, activation=tanh
13번째 컬럼: dim=15, activation=softmax
14번째 컬럼: dim=1, activation=tanh
15번째 컬럼: dim=13, activation=softmax
16번째 컬럼: dim=1, activation=tanh
17번째 컬럼: dim=15, activation=softmax
18번째 컬럼: dim=2, activation=softmax
19번째 컬럼: dim=35, activation=softmax
20번째 컬럼: dim=6, activation=softmax
21번째 컬럼: dim=14, activation=softmax
22번째 컬럼: dim=7, activation=softmax
23번째 컬럼: dim=1, activation=tanh
24번째 컬럼: dim=14, activation=softmax
25번째 컬럼: dim=1, activation=tanh
26번째 컬럼: dim=14, activation=softmax
27번째 컬럼: dim=1, activation=tanh
28번째 컬럼: dim=15, activation=softmax
29번째 컬럼: dim=1, a